In [1]:
%load_ext autoreload

In [2]:
#from cap_package import SpotipyCollectPub as scp
from cap_package import ReadTransform as rt
from datetime import datetime
from dotenv import load_dotenv
import pandas as pd
import numpy as np
import os
from pathlib import Path

In [3]:
%autoreload 2

In [4]:
from IPython.utils.text import columnize
def disp_col(list_):
    
    # import -> from IPython.utils.text import columnize
    l = list(map(lambda x:repr(x)+',', list_))
    print(columnize(l, displaywidth=120))
def timer(start_time=None):
    if start_time is not None:
        return (datetime.now() - start_time).total_seconds()
    else:
        start_time = datetime.now()
        return start_time

In [5]:
load_dotenv()
path = Path(os.getenv('PATH_DATASET1.2'))

In [6]:
timbre_ = ['timbre_{}'.format(i+1) for i in range(12)]
pitch_ = ['pitch_{}'.format(i+1) for i in range(12)]

In [7]:
# Read dataset

path_ = path.joinpath('user_playlists')
pl_dataset = rt.read_dataset(path_)

print('Number of playlists:', len(pl_dataset))

Number of playlists: 13


In [8]:
# create data label list from playlist names
data_labels = []

for pl in pl_dataset:
   
    # extract playlist name
    name_ = pl[0] 
    # Number of tracks in this playlist
    num = len(pl[1]) 
    # create same labels for all tracks in this playlist
    # add to dataset labels
    data_labels += [name_] * len(pl[1])

print('\nFollowing playlists are in the dataset:\n')
disp_col(set(data_labels))


Following playlists are in the dataset:

'Progressive 3',          'That familiar trance',     'Progressive 5',  'Progressive House',          'Progressive 4',
'Our old school trance',  'Our old school trance 2',  'Progressive 2',  'Deep house',               
'House-Trance',           'Classic progressive',      'Progressive 1',  'Our old school trance 138',



In [9]:
# spilt columns for pitch and timbre vectors in segment dataframes
# Make separate list of track names and updated segments dataframes
tracks_name = []
tracks_seg = []
for pl in pl_dataset:
    
        for track in pl[1]:
            tracks_name.append(track[0])
            tracks_seg.append(rt.split_columns(track[1], pitch_cols=pitch_, timbre_cols=timbre_, ))

In [10]:
# remove any commas in track names; we will be using ',' as the delimiter for saving names in a csv
for i in range(len(tracks_name)):
    
    tracks_name[i] = tracks_name[i].replace(',', '')

In [11]:
if len(tracks_name) == len(tracks_seg) == len(data_labels):
    print("all good..............")

all good..............


In [12]:
# Check for null values in segments dataframe

nulls_in = []
for i in  range(len(tracks_seg)):
    n = tracks_seg[i].isnull().sum().sum()
    if n > 0:
        nulls_in.append(i)

In [15]:
dfs =[]
for t in pl_dataset[0][1]:
    dfs.append(t[1])

In [16]:
tracks_name[0]

'2 Roads [Mix Cut] - Blood Groove & Kikis Remix_Tre'

In [58]:
np.argsort(np.argsort(dfs[0]['pitches'][16]))

array([ 8, 11, 10,  9,  5,  0,  4,  1,  3,  7,  6,  2], dtype=int64)

In [68]:
top_pitches = []
for df in dfs:
    mask_ = df['pitches'].map(lambda x: np.argsort(np.argsort(x))>6)
    
    mask_ = pd.DataFrame(mask_.tolist(), index=mask_.index)
    
    top_pitches.append(np.argsort(np.argsort(np.array(mask_.sum(axis=0)))))

In [69]:
top_pitches

[array([ 7, 11,  9, 10,  2,  6,  4,  0,  1,  8,  5,  3], dtype=int64),
 array([ 1, 11,  2,  8,  0,  4,  9,  6, 10,  5,  7,  3], dtype=int64),
 array([ 9,  4,  6,  0, 11,  1,  2,  3,  7, 10,  5,  8], dtype=int64),
 array([11, 10,  1,  2,  5,  7,  6,  8,  9,  0,  4,  3], dtype=int64),
 array([10,  8,  9,  4,  6,  2,  0,  5,  3,  7,  1, 11], dtype=int64),
 array([11,  9,  8,  3,  4,  5,  0,  6,  2,  7,  1, 10], dtype=int64),
 array([11,  1,  2,  3, 10,  5,  9,  6,  8,  4,  0,  7], dtype=int64),
 array([10, 11,  7,  9,  5,  8,  0,  3,  2,  6,  1,  4], dtype=int64),
 array([ 8,  9,  5,  2,  1,  7, 11,  4,  3,  0,  6, 10], dtype=int64),
 array([ 7,  2,  3,  1, 10,  5,  6,  9,  4,  8,  0, 11], dtype=int64),
 array([ 9, 11,  5,  2,  1,  0,  7,  4,  8,  6,  3, 10], dtype=int64),
 array([ 8, 11,  1,  2,  0, 10,  7,  5,  6,  3,  9,  4], dtype=int64),
 array([ 3,  1,  5,  7,  0,  9,  2, 11,  4,  8, 10,  6], dtype=int64),
 array([ 9, 10,  6,  8,  7,  5,  3,  4,  0,  1,  2, 11], dtype=int64),
 array

In [ ]:
# Make a mins list and a max list of timbre elements of each track

timbre_mins = []
timbre_maxs = []

for tr_seg in tracks_seg:
    
    min_ , max_ = rt.timbre_minmax_tr(tr_seg)
    timbre_mins.append(min_)
    timbre_maxs.append(max_)

In [ ]:
# Evaluate population min and max of timbre elements

pop_timbre_min, pop_timbre_max = rt.pop_timbre_minmax(timbre_mins, timbre_maxs)

In [ ]:
if len(timbre_mins) == len(timbre_maxs):
    print('Total number of tracks : ', len(timbre_mins))

if nulls_in:
    print('\nCheck "nulls_in" to see which tracks have missing values')
else:
    print('\nThere are no missing values in the dataset')
    
print('\nMinimum of timbre vec values for a sample track :\n ', timbre_mins[0])   
print('\nMaximum of timbre vec values for a sample track :\n ', timbre_maxs[0]) 

print('\nMinimums of timbre values over the whole dataset: \n{}'.format(pop_timbre_min))
print('\nMaximums of timbre values over the whole dataset: \n{}'.format(pop_timbre_max))

In [ ]:
data_arrays = rt.transform_dataset(tracks_seg, pop_timbre_min, pop_timbre_max, num_seg=100, bin_num=10)

In [ ]:
print('Number of tracks :', len(data_arrays))
print('Length of an input array :', len(data_arrays[0]))
print('First 100 elements of a sample input array :\n')
disp_col(list(data_arrays[0][:100]))

In [ ]:
data_y, categories = rt.encode_label(data_labels)

In [ ]:
categories

In [ ]:
data = [i for i in zip(data_arrays, data_y)]

In [ ]:
#p = path.joinpath('usertracks_csv')
np.savetxt(p.joinpath('track_names.csv'), tracks_name, fmt='%s', delimiter = ',')
np.savetxt(p.joinpath('label_categories.csv'), categories, fmt='%s', delimiter = ',')

In [ ]:
#p1 = p.joinpath('track_arrays') 
for t in range(len(data)):
    np.savetxt(p1.joinpath('track_inp_{}.csv'.format(t)), data[t][0], fmt='%+10.3f', delimiter = ',')
    
#p2 = p.joinpath('track_labels') 
for t in range(len(data)):
    np.savetxt(p2.joinpath('track_lab_{}.csv'.format(t)), data[t][1], fmt='%1.1f', delimiter = ',')
